# JUPITER NOTEBOOK #

Il dataset è così composto:
Ogni riga in ogni file ha un id univoco, che può essere utilizzato per collegare i file tra loro.
Ogni id si riferisce ad un film diverso, con crew, attori, generi, paesi, lingue, etc.


### Import delle librerie ###

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import numpy as np

### Caricamento dei dati ###

In [2]:

# Percorsi dei main file
data_path_actors = os.path.join('Data', 'Main', 'actors.csv')
data_path_countries = os.path.join('Data', 'Main', 'countries.csv')
data_path_crews = os.path.join('Data', 'Main', 'crew.csv')
data_path_genres = os.path.join('Data', 'Main', 'genres.csv')
data_path_languages = os.path.join('Data', 'Main', 'languages.csv')
data_path_movies = os.path.join('Data', 'Main', 'movies.csv')
data_path_posters = os.path.join('Data', 'Main', 'posters.csv')
data_path_releases = os.path.join('Data', 'Main', 'releases.csv')
data_path_studios = os.path.join('Data', 'Main', 'studios.csv')
data_path_themes = os.path.join('Data', 'Main', 'themes.csv')

# Percorsi dei side file
data_path_rotten_tomatoes = os.path.join('Data', 'Side', 'rotten_tomatoes_reviews.csv')
data_path_oscar = os.path.join('Data', 'Side', 'the_oscar_awards.csv')

# Carica i dataframe
actors_df = pd.read_csv(data_path_actors)
countries_df = pd.read_csv(data_path_countries)
crews_df = pd.read_csv(data_path_crews)
genres_df = pd.read_csv(data_path_genres)
languages_df = pd.read_csv(data_path_languages)
movies_df = pd.read_csv(data_path_movies)
posters_df = pd.read_csv(data_path_posters)
releases_df = pd.read_csv(data_path_releases)
studios_df = pd.read_csv(data_path_studios)
themes_df = pd.read_csv(data_path_themes)

rotten_tomatoes_df = pd.read_csv(data_path_rotten_tomatoes)
oscar_df = pd.read_csv(data_path_oscar)

## Analisi di Actors ##

Actors rappresenta per ogni film (rappresentato dal suo id) il nome dell'attore e il ruolo che ha interpretato

In [14]:
print(actors_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5798450 entries, 0 to 5798449
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   name    object
 2   role    object
dtypes: int64(1), object(2)
memory usage: 132.7+ MB
None


### Controllo di attori nulli ###

In [15]:
print(actors_df.isna().sum())

id            0
name          4
role    1361559
dtype: int64


Abbiamo degli attori di cui è presente l'id del film a cui hanno partecipato ma non il loro nome. Questi attori verranno rimossi.
Ecco suddetti attori:

In [3]:
# Print rows with null 'name'
null_name_rows = actors_df[actors_df['name'].isna()]
print("Rows with null 'name':")
print(null_name_rows)

Rows with null 'name':
              id name            role
4145738  1443629  NaN             NaN
4281100  1469981  NaN            Self
4306960  1474958  NaN  Cinematography
5430275  1773264  NaN             NaN


Non sono molto utili nell'analisi quindi si sceglie di rimuoverli

In [17]:
# Remove rows with null 'name'
actors_df = actors_df.dropna(subset=['name'])

# Check if there are any more null 'name' values
null_name_count = actors_df['name'].isna().sum()
if null_name_count == 0:
    print("No rows with null 'name' remain.")
else:
    print(f"There are still {null_name_count} rows with null 'name'.")

No rows with null 'name' remain.


### Rimozione Attori Duplicati ###

Per il criterio di rimozione dei duplicati, si è scelto di mantenere la prima occorrenza di ogni attore duplicato. Un attore duplicato ha tutti i campi uguali dato che ad esempio un attore può avere più ruoli nello stesso film, o lo stesso ruolo in due film diversi (es. sequel di film).

#### Stampa degli attori duplicati ####

In [5]:
# Create a copy of the original DataFrame
actors_df_copy = actors_df.copy()

# Create a new column 'duplicate_count' with the number of duplicates for each row
actors_df_copy['duplicate_count'] = actors_df_copy.groupby(['name', 'role', 'id'])['name'].transform('count')

# Filter rows where 'duplicate_count' is greater than 1
duplicates_df = actors_df_copy[actors_df_copy['duplicate_count'] > 1]

# Print the duplicate actors and the number of duplicates
print("Duplicate actors:")
duplicates_df[['name', 'role', 'id', 'duplicate_count']]


Duplicate actors:


,name,role,id,duplicate_count
3967,Rosie Jones,Lady of the Boot of Jemiah,1000062,2.0
3993,Rosie Jones,Lady of the Boot of Jemiah,1000062,2.0
44615,Karel Heřmánek,Czech Injured Man,1000797,2.0
44642,Karel Heřmánek,Czech Injured Man,1000797,2.0
47806,Michael Fennimore,Car Salesman,1000863,2.0
...,...,...,...,...
5788273,Matthew Evancic,Don Montana (voice),1935866,3.0
5792882,Gudni Oddgeirsson,Interviewee,1939290,2.0
5792884,Gudni Oddgeirsson,Interviewee,1939290,2.0
5795227,Ann Victorin,other voices,1940468,2.0


Come possiamo notare ci sono attori duplicati anche 3 volte, quindi procediamo con la rimozione.

In [6]:
# Counting the total number of duplicates
total_duplicates = duplicates_df['duplicate_count'].sum()
print(f"Total number of duplicates: {total_duplicates}")

Total number of duplicates: 2133.0


Questa operazione permette di alleggerire il dataset e di avere una visione più precisa dei dati.

##### Rimozione degli attori duplicati #####

In [24]:
# Rimuovi i duplicati
actors_df = actors_df.drop_duplicates(subset=['name', 'role', 'id'], keep='first')

# Controlla se ci sono ancora duplicati
remaining_duplicates = actors_df.duplicated(subset=['name', 'role', 'id']).sum()

# Stampa il risultato
if remaining_duplicates == 0:
    print("Non ci sono duplicati rimanenti.")
else:
    print(f"Ci sono ancora {remaining_duplicates} duplicati rimanenti.")

Non ci sono duplicati rimanenti.


## Analisi di Countries ##

Countries non ha bisogno di pulizia in quanto non ci sono valori nulli, duplicati o mancanti.

In [26]:
print(countries_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693476 entries, 0 to 693475
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       693476 non-null  int64 
 1   country  693476 non-null  object
dtypes: int64(1), object(1)
memory usage: 10.6+ MB
None


In [7]:
# Controlla se ci sono duplicati nel DataFrame 'countries_df'
duplicate_rows = countries_df[countries_df.duplicated()]

# Stampa le righe duplicate
if not duplicate_rows.empty:
    print("Righe duplicate trovate:")
    print(duplicate_rows)
else:
    print("Non ci sono duplicati nel DataFrame 'countries_df'.")

Non ci sono duplicati nel DataFrame 'countries_df'.


## Analisi di Crews ##

Crews rappresenta per ogni film (rappresentato dal suo id) il nome della persona coinvolta nella produzione del film e il ruolo che ha ricoperto.

### Controllo di Crews ###

In [28]:
print(crews_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4720183 entries, 0 to 4720182
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   role    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 108.0+ MB
None


Crews contiene tre colonne, che sono id (riferimento al film), name (nome della persona coinvolta nella produzione del film) e role (ruolo che ha ricoperto).

### Controllo elementi di Crews Nulli ###

In [29]:
print(crews_df.isna().sum())

id      0
role    0
name    1
dtype: int64


Abbiamo un solo record con 'name' nullo, che verrà rimosso perchè potrebbe essere inserito nel dataset successivamente in modo corretto.

### Stampa elementi di Crews Nulli ###

In [31]:
# Print rows with null 'name'
null_name_rows = crews_df[crews_df['name'].isna()]

print("Rows with null 'name':")
null_name_rows

Rows with null 'name':


,id,role,name
4562126,1859397,Writer,NaN


### Rimozione elementi di Crews Nulli ###

In [8]:
# Remove rows with null 'name'
crews_df = crews_df.dropna(subset=['name'])

# Check if there are any more null 'name' values
null_name_count = crews_df['name'].isna().sum()
if null_name_count == 0:
    print("No rows with null 'name' remain.")
else:
    print(f"There are still {null_name_count} rows with null 'name'.")


No rows with null 'name' remain.


### Rimozione Crews Duplicati ###

In [33]:
# Create a copy of the original DataFrame
crews_df_copy = crews_df.copy()

# Create a new column 'duplicate_count' with the number of duplicates for each row
crews_df_copy['duplicate_count'] = crews_df_copy.groupby(['name', 'role', 'id'])['name'].transform('count')

# Filter rows where 'duplicate_count' is greater than 1
duplicates_df = crews_df_copy[crews_df_copy['duplicate_count'] > 1]

# Print the duplicate crews and the number of duplicates
print("Duplicate crews:")
duplicates_df[['name', 'role', 'id', 'duplicate_count']]

Duplicate crews:


,name,role,id,duplicate_count
1680,Chris Webb,Stunts,1000018,2
1721,Chris Webb,Stunts,1000018,2
2690,Sarah Irwin,Stunts,1000031,3
2691,Sarah Irwin,Stunts,1000031,3
2692,Sarah Irwin,Stunts,1000031,3
...,...,...,...,...
4718342,Choe Yeong-sik,Assistant director,1940904,2
4719627,Josh Earl,Executive producer,1941357,2
4719628,Josh Earl,Executive producer,1941357,2
4720108,Oscar van Hoogevest,Sound,1941521,2


Come possiamo notare ci sono crew duplicati anche 3 volte, quindi procediamo con la rimozione.

In [34]:
# Rimuovi i duplicati
crews_df = crews_df.drop_duplicates(subset=['name', 'role', 'id'], keep='first')

# Controlla se ci sono ancora duplicati
remaining_duplicates = crews_df.duplicated(subset=['name', 'role', 'id']).sum()

# Stampa il risultato
if remaining_duplicates == 0:
    print("Non ci sono duplicati rimanenti.")
else:
    print(f"Ci sono ancora {remaining_duplicates} duplicati rimanenti.")


Non ci sono duplicati rimanenti.


## Analisi di Genres ##

Genres rappresenta per ogni film (rappresentato dal suo id) il genere a cui appartiene. Per un film ci possono essere più generi, questo dipende dal tipo di film, un esempio di generi può essere "Azione, Avventura, Commedia" ecc...

In [35]:
print(genres_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046849 entries, 0 to 1046848
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id      1046849 non-null  int64 
 1   genre   1046849 non-null  object
dtypes: int64(1), object(1)
memory usage: 16.0+ MB
None


Come possiamo vedere abbiamo due colonne, id (riferimento al film) e genre (genere del film).

In [36]:
print(genres_df.isna().sum())

id       0
genre    0
dtype: int64


Non abbiamo valori nulli da rimuovere.

In [38]:
# Create a copy of the original DataFrame
genres_df_copy = genres_df.copy()

# Create a new column 'duplicate_count' with the number of duplicates for each row
genres_df_copy['duplicate_count'] = genres_df_copy.groupby(['genre', 'id'])['genre'].transform('count')

# Filter rows where 'duplicate_count' is greater than 1
duplicates_df = genres_df_copy[genres_df_copy['duplicate_count'] > 1]

# Print the duplicate genres and the number of duplicates
print("Duplicate genres:")
duplicates_df[['genre', 'id', 'duplicate_count']]

Duplicate genres:


,genre,id,duplicate_count


Non ci sono nè duplicati, nè entry inserite male da rimuovere.

## Analisi di Languages ##

Languages rappresenta per ogni film (rappresentato dal suo id) la lingua in cui è stato girato. Per un film ci possono essere più lingue, questo dipende dal tipo di film, un esempio di lingue può essere "Inglese, Italiano, Francese" ecc...
Abbiamo varie distinzioni delle lingue usate in un film, ad esempio type include "Primary language", "Spoken language" o "Language" più nel generico. Ad esempio Parasite (id 1000002) ha come lingua originale il coreano e come lingua parlata l'inglese e il tedesco. Queste sono lingue parlate nel film originale e non si riferiscono agli adattamenti.

In [40]:
print(languages_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038762 entries, 0 to 1038761
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   id        1038762 non-null  int64 
 1   type      1038762 non-null  object
 2   language  1038762 non-null  object
dtypes: int64(1), object(2)
memory usage: 23.8+ MB
None


In [41]:
print(languages_df.isna().sum())

id          0
type        0
language    0
dtype: int64


Non abbiamo valori nulli da rimuovere.

In [44]:
# Create a copy of the original DataFrame
languages_df_copy = languages_df.copy()

# Create a new column 'duplicate_count' with the number of duplicates for each row
languages_df_copy['duplicate_count'] = languages_df_copy.groupby(['language','type','id'])['language'].transform('count')

# Filter rows where 'duplicate_count' is greater than 1
duplicates_df = languages_df_copy[languages_df_copy['duplicate_count'] > 1]

# Print the duplicate languages and the number of duplicates
print("Duplicate languages:")
duplicates_df[['language', 'type', 'id', 'duplicate_count']]

Duplicate languages:


,language,type,id,duplicate_count


Non ci sono duplicati da rimuovere.

## Analisi di Movies ##

Movies è il dataframe principale a cui si collegano tutti gli altri attraverso la colonna id. Contiene informazioni sui film come il titolo, la data di uscita, la durata, il rating e il budget speso nella creazione. Queste sono ritenute le informazioni più importanti dal creatore del set.

In [46]:
print(movies_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941597 entries, 0 to 941596
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           941597 non-null  int64  
 1   name         941587 non-null  object 
 2   date         849684 non-null  float64
 3   tagline      139387 non-null  object 
 4   description  780785 non-null  object 
 5   minute       760027 non-null  float64
 6   rating       90999 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 50.3+ MB
None


Come possiamo notare abbiamo molte colonne con valori nulli, alcune non sono considerate essenziali per il riconoscimento del film, ma servono per la successiva analisi di essi. Ad esempio rating ha molti valori nulli, che lasceremo nulli perchè riempirli con la media di tutti gli altri film costituisce una distorsione del dato singolo. Inoltre sarebbe più difficile successivamente riconoscere i film con rating nulli e all'inserimento di nuovi film sarebbe da ricalcolare la media.

Abbiamo 10 film senza nome e valori nulli in altre colonne, ma che non rimuoveremo perché potrebbero essere utili come informazioni parziali. In futuro colonne problematiche come date, minute e rating verranno considerate.

In [47]:
print(movies_df.isna().sum())

id                  0
name               10
date            91913
tagline        802210
description    160812
minute         181570
rating         850598
dtype: int64


In [50]:
# Print rows with null 'name'
null_title_rows = movies_df[movies_df['name'].isna()]
print("Rows with null 'name':")
null_title_rows

Rows with null 'name':


,id,name,date,tagline,description,minute,rating
287514,1287515,NaN,2015.0,NaN,NONE is a short film that explores the balance...,4.0,NaN
617642,1617643,NaN,NaN,NaN,NaN,NaN,NaN
646520,1646521,NaN,2008.0,NaN,NaN,NaN,NaN
648185,1648186,NaN,NaN,NaN,NaN,NaN,NaN
720294,1720295,NaN,NaN,NaN,"In this directorial debut of Eden Ewardson, he...",8.0,NaN
725369,1725370,NaN,NaN,NaN,NaN,NaN,NaN
741481,1741482,NaN,NaN,NaN,NaN,90.0,NaN
840337,1840338,NaN,NaN,NaN,NaN,NaN,NaN
883228,1883229,NaN,NaN,NaN,NaN,NaN,NaN
894771,1894772,NaN,NaN,NaN,NaN,NaN,NaN


Ecco un esempio di film senza nome, che verrà rimosso in quanto non è possibile identificarlo, inoltre potrebbe interferire con la statistica di altri film.

In [51]:
# Remove rows with null 'name'
movies_df = movies_df.dropna(subset=['name'])

# Check if there are any more null 'name' values
null_name_count = movies_df['name'].isna().sum()
if null_name_count == 0:
    print("No rows with null 'name' remain.")
else:
    print(f"There are still {null_name_count} rows with null 'name'.")

No rows with null 'name' remain.


In [54]:
# Create a copy of the original DataFrame
movies_df_copy = movies_df.copy()

# Create a new column 'duplicate_count' with the number of duplicates for each row
movies_df_copy['duplicate_count'] = movies_df_copy.groupby(['id'])['id'].transform('count')

# Filter rows where 'duplicate_count' is greater than 1
duplicates_df = movies_df_copy[movies_df_copy['duplicate_count'] > 1]

# Print the duplicate movies and the number of duplicates
print("Duplicate movies:")
duplicates_df[['id','name', 'date', 'rating', 'minute', 'duplicate_count']]

Duplicate movies:


,id,name,date,rating,minute,duplicate_count


Non ci sono film duplicati da rimuovere.

## Analisi di Posters ##


Posters è un frame che rappresenta per ogni film (rappresentato dal suo id) il link al poster del film, questo è utile per una visualizzazione migliorata dei dati ad esempio nell'implementazione di un sito web.

In [55]:
print(posters_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941597 entries, 0 to 941596
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      941597 non-null  int64 
 1   link    760885 non-null  object
dtypes: int64(1), object(1)
memory usage: 14.4+ MB
None


In [56]:
print(posters_df.isna().sum())

id           0
link    180712
dtype: int64


Ci sono alcuni film senza poster, non li rimuoveremo perchè in un secondo momento potrebbero essere aggiunti usando l'id per la ricerca di essi.

In [58]:
posters_df_copy = posters_df.copy()
posters_df_copy['duplicate_count'] = posters_df_copy.groupby(['id'])['id'].transform('count')
duplicates_df = posters_df_copy[posters_df_copy['duplicate_count'] > 1]
print("Duplicate posters:")
duplicates_df[['id', 'duplicate_count']]


Duplicate posters:


,id,duplicate_count


Non ci sono poster duplicati da rimuovere.

## Analisi di Releases ##

In [59]:
print(releases_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332782 entries, 0 to 1332781
Data columns (total 5 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   id       1332782 non-null  int64 
 1   country  1332782 non-null  object
 2   date     1332782 non-null  object
 3   type     1332782 non-null  object
 4   rating   333980 non-null   object
dtypes: int64(1), object(4)
memory usage: 50.8+ MB
None


In [60]:
print(releases_df.isna().sum())

id              0
country         0
date            0
type            0
rating     998802
dtype: int64


I ratings non sono obbligatori, dato che variano da paese a paese. Non li rimuoveremo.

In [63]:
# Checking for duplicates
releases_df_copy = releases_df.copy()
releases_df_copy['duplicate_count'] = releases_df_copy.groupby(['id', 'country', 'date','type'])['id'].transform('count')
duplicates_df = releases_df_copy[releases_df_copy['duplicate_count'] > 1]
print("Duplicate releases:")
duplicates_df[['id', 'country', 'date', 'duplicate_count']]

Duplicate releases:


,id,country,date,duplicate_count


Non ci sono duplicati da rimuovere.

## Analisi di Studios ##

In [64]:
print(studios_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679283 entries, 0 to 679282
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      679283 non-null  int64 
 1   studio  679273 non-null  object
dtypes: int64(1), object(1)
memory usage: 10.4+ MB
None


In [65]:
print(studios_df.isna().sum())

id         0
studio    10
dtype: int64


Abbiamo 10 studios senza nome, ma che in futuro potrebbero essere utili come informazioni parziali. Rinominiamo gli studios senza nome in 'Unknown'.

In [68]:
# Fill missing values in 'name' with 'Unknown'
studios_df['studio'] = studios_df['studio'].fillna('Unknown')

In [69]:
# Checking for duplicates
studios_df_copy = studios_df.copy()
studios_df_copy['duplicate_count'] = studios_df_copy.groupby(['studio', 'id'])['studio'].transform('count')
duplicates_df = studios_df_copy[studios_df_copy['duplicate_count'] > 1]
print("Duplicate studios:")
duplicates_df[['studio', 'id', 'duplicate_count']]

Duplicate studios:


,studio,id,duplicate_count
145,Working Title Films,1000044,2
146,Working Title Films,1000044,2
485,Working Title Films,1000165,2
487,Working Title Films,1000165,2
809,Working Title Films,1000263,2
...,...,...,...
656807,Star Media,1863229,2
665452,Deutsche Film- und Fernsehakademie Berlin (DFFB),1888903,2
665454,Deutsche Film- und Fernsehakademie Berlin (DFFB),1888903,2
677099,Ministerstvo kultury ČR,1934998,2


In [70]:
# Remove duplicates
studios_df = studios_df.drop_duplicates(subset=['studio', 'id'], keep='first')

# Check if there are any duplicates left
remaining_duplicates = studios_df.duplicated(subset=['studio', 'id']).sum()

# Print the result
if remaining_duplicates == 0:
    print("No duplicates remain.")
else:
    print(f"There are still {remaining_duplicates} duplicates remaining.")

No duplicates remain.


## Analisi di Themes ##

In [72]:
print(themes_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125641 entries, 0 to 125640
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      125641 non-null  int64 
 1   theme   125641 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.9+ MB
None


In [73]:
print(themes_df.isna().sum())

id       0
theme    0
dtype: int64


In [76]:
# Checking for duplicates
themes_df_copy = themes_df.copy()
themes_df_copy['duplicate_count'] = themes_df_copy.groupby(['theme', 'id'])['theme'].transform('count')
duplicates_df = themes_df_copy[themes_df_copy['duplicate_count'] > 1]
print("Duplicate themes:")
duplicates_df[['theme', 'id', 'duplicate_count']]

Duplicate themes:


,theme,id,duplicate_count


Non ci sono duplicati.

## Analisi di Rotten Tomatoes Reviews##

In [5]:
print(rotten_tomatoes_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1129887 entries, 0 to 1129886
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   rotten_tomatoes_link  1129887 non-null  object
 1   movie_title           1129887 non-null  object
 2   critic_name           1111366 non-null  object
 3   top_critic            1129887 non-null  bool  
 4   publisher_name        1129887 non-null  object
 5   review_type           1129887 non-null  object
 6   review_score          823985 non-null   object
 7   review_date           1129887 non-null  object
 8   review_content        1064109 non-null  object
dtypes: bool(1), object(8)
memory usage: 70.0+ MB
None


In [6]:
print(rotten_tomatoes_df.isna().sum())

rotten_tomatoes_link         0
movie_title                  0
critic_name              18521
top_critic                   0
publisher_name               0
review_type                  0
review_score            305902
review_date                  0
review_content           65778
dtype: int64


In [7]:
rotten_tomatoes_df.head(20)


,rotten_tomatoes_link,movie_title,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,Percy Jackson & the Olympians: The Lightning T...,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...
5,m/0814255,Percy Jackson & the Olympians: The Lightning T...,David Germain,True,Associated Press,Rotten,NaN,2010-02-10,It's more a list of ingredients than a movie-m...
6,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Nick Schager,False,Slant Magazine,Rotten,1/4,2010-02-10,Harry Potter knockoffs don't come more transpa...
7,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Bill Goodykoontz,True,Arizona Republic,Fresh,3.5/5,2010-02-10,"Percy Jackson isn't a great movie, but it's a ..."
8,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Jordan Hoffman,False,UGO,Fresh,B,2010-02-10,"Fun, brisk and imaginative"
9,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Jim Schembri,True,The Age (Australia),Fresh,3/5,2010-02-10,"Crammed with dragons, set-destroying fights an..."


### Metodo di conversione del punteggio in percentuale ###

In [27]:
def convert_fraction_to_percentage(score):
    try:
        num, denom = map(float, score.split('/'))
        if denom > 0 and num <= denom:
            return round((num / denom) * 100,2)
        else:
            return np.nan
    except:
        return np.nan

def convert_letter_to_percentage(score):
    letter_to_percentage = {
        'A': 100,
        'A-': 90,
        'B+': 87,
        'B': 83,
        'B-': 80,
        'C+': 77,
        'C': 73,
        'C-': 70,
        'D+': 67,
        'D': 63,
        'D-': 60,
        'F': 50
    }
    return letter_to_percentage.get(score, np.nan)

def convert_to_percentage(score):
    if isinstance(score, str) and '/' in score:
        return convert_fraction_to_percentage(score)
    elif isinstance(score, str) and score in ['A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F']:
        return convert_letter_to_percentage(score)
    else:
        return np.nan

### Conversione del punteggio in percentuale ###

In [28]:
# Assuming 'review_score' is the column with the review scores
rotten_tomatoes_df['review_score_percentage'] = rotten_tomatoes_df['review_score'].apply(convert_to_percentage)

# Print the DataFrame to check the results
rotten_tomatoes_df[['movie_title', 'review_score', 'review_score_percentage']]

#rotten_tomatoes_df.loc[rotten_tomatoes_df['review_score']=='C+', ['movie_title', 'review_score', 'review_score_percentage']]

,movie_title,review_score,review_score_percentage
0,Percy Jackson & the Olympians: The Lightning T...,NaN,NaN
1,Percy Jackson & the Olympians: The Lightning T...,NaN,NaN
2,Percy Jackson & the Olympians: The Lightning T...,NaN,NaN
3,Percy Jackson & the Olympians: The Lightning T...,3.5/5,70.0
4,Percy Jackson & the Olympians: The Lightning T...,NaN,NaN
...,...,...,...
1129882,Zulu Dawn,2/5,40.0
1129883,Zulu Dawn,3.5/5,70.0
1129884,Zulu Dawn,B+,87.0
1129885,Zulu Dawn,3.5/5,70.0


In [30]:
# Define the fields to check for duplicates
fields = ['rotten_tomatoes_link', 'movie_title', 'critic_name', 'top_critic', 'publisher_name', 'review_type', 'review_score', 'review_date', 'review_content']

# Find duplicate rows based on the specified fields
duplicates_df = rotten_tomatoes_df[rotten_tomatoes_df.duplicated(subset=fields, keep=False)]

# Print the duplicate rows
print("Duplicate reviews based on specified fields:")
duplicates_df

Duplicate reviews based on specified fields:


,rotten_tomatoes_link,movie_title,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,review_score_percentage
35513,m/1069696-screamers,Screamers,Dave White,False,Movies.com,Fresh,B-,1996-01-26,NaN,80.0
35514,m/1069696-screamers,Screamers,Dave White,False,Movies.com,Fresh,B-,1996-01-26,NaN,80.0
35576,m/1069707-othello,Othello,Fred Topel,False,About.com,Fresh,4/5,2003-11-25,Fine Shakespeare adaptation,80.0
35577,m/1069707-othello,Othello,Fred Topel,False,About.com,Fresh,4/5,2003-11-25,Fine Shakespeare adaptation,80.0
41315,m/1087935-buena_vista_social_club,Buena Vista Social Club,NaN,False,Film Threat,Fresh,4/5,2002-12-08,NaN,80.0
...,...,...,...,...,...,...,...,...,...,...
955042,m/the_fog_of_war_eleven_lessons_from_the_life_...,The Fog of War: Eleven Lessons from the Life o...,NaN,False,Film Threat,Fresh,4/5,2005-12-06,NaN,80.0
959360,m/the_girl_with_the_dragon_tattoo_2009,The Girl with the Dragon Tattoo,NaN,False,National Post,Fresh,3.5/4,2009-02-27,NaN,87.5
959361,m/the_girl_with_the_dragon_tattoo_2009,The Girl with the Dragon Tattoo,NaN,False,National Post,Fresh,3.5/4,2009-02-27,NaN,87.5
1044907,m/together_2001,Together,NaN,False,Film Threat,Fresh,4/5,2002-12-08,NaN,80.0


In [31]:
# Remove duplicates
rotten_tomatoes_df = rotten_tomatoes_df.drop_duplicates(subset=fields, keep='first')

# Check if there are any duplicates left
remaining_duplicates = rotten_tomatoes_df.duplicated(subset=fields).sum()

# Print the result
if remaining_duplicates == 0:
    print("No duplicates remain.")
else:
    print(f"There are still {remaining_duplicates} duplicates remaining.")

No duplicates remain.


## Analisi di The Oscar Awards ##

In [32]:
print(oscar_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10889 entries, 0 to 10888
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      10889 non-null  int64 
 1   year_ceremony  10889 non-null  int64 
 2   ceremony       10889 non-null  int64 
 3   category       10889 non-null  object
 4   name           10884 non-null  object
 5   film           10570 non-null  object
 6   winner         10889 non-null  bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 521.2+ KB
None


In [33]:
print(oscar_df.isna().sum())

year_film          0
year_ceremony      0
ceremony           0
category           0
name               5
film             319
winner             0
dtype: int64


In [35]:
#Stampo i valori nulli di name
null_name_rows = oscar_df[oscar_df['name'].isna()]
print("Rows with null 'name':")
null_name_rows

Rows with null 'name':


,year_film,year_ceremony,ceremony,category,name,film,winner
10513,2020,2021,93,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True
10514,2020,2021,93,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True
10635,2021,2022,94,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True
10759,2022,2023,95,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True
10885,2023,2024,96,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True


In [39]:
# Check for duplicates
oscar_df_copy = oscar_df.copy()
oscar_df_copy['duplicate_count'] = oscar_df_copy.groupby(['name','film', 'year_ceremony','year_film', 'ceremony', 'category', 'winner'])['name'].transform('count')
duplicates_df = oscar_df_copy[oscar_df_copy['duplicate_count'] > 1]
print("Duplicate Oscar awards:")
duplicates_df[['duplicate_count','name','film', 'year_ceremony','year_film', 'ceremony', 'category', 'winner']]

Duplicate Oscar awards:


,duplicate_count,name,film,year_ceremony,year_film,ceremony,category,winner
6219,2.0,Music by Michel Legrand; Lyric by Alan Bergman...,Yentl,1984,1983,56,MUSIC (Original Song),False
6220,2.0,Music by Michel Legrand; Lyric by Alan Bergman...,Yentl,1984,1983,56,MUSIC (Original Song),False
7066,2.0,Music by Alan Menken; Lyric by Howard Ashman,Beauty and the Beast,1992,1991,64,MUSIC (Original Song),False
7068,2.0,Music by Alan Menken; Lyric by Howard Ashman,Beauty and the Beast,1992,1991,64,MUSIC (Original Song),False
7394,2.0,Music by Elton John; Lyric by Tim Rice,The Lion King,1995,1994,67,MUSIC (Original Song),False
7395,2.0,Music by Elton John; Lyric by Tim Rice,The Lion King,1995,1994,67,MUSIC (Original Song),False
8862,3.0,Music by Alan Menken; Lyric by Stephen Schwartz,Enchanted,2008,2007,80,MUSIC (Original Song),False
8864,3.0,Music by Alan Menken; Lyric by Stephen Schwartz,Enchanted,2008,2007,80,MUSIC (Original Song),False
8865,3.0,Music by Alan Menken; Lyric by Stephen Schwartz,Enchanted,2008,2007,80,MUSIC (Original Song),False
9090,2.0,Music and Lyric by Randy Newman,The Princess and the Frog,2010,2009,82,MUSIC (Original Song),False


In [40]:
# Remove duplicates
oscar_df = oscar_df.drop_duplicates(subset=['name','film', 'year_ceremony','year_film', 'ceremony', 'category', 'winner'], keep='first')

In [41]:
# Remove rows with null 'name' and 'film'
oscar_df = oscar_df.dropna(subset=['name', 'film'])
oscar_df

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10880,2023,2024,96,WRITING (Original Screenplay),Screenplay - Justine Triet and Arthur Harari,Anatomy of a Fall,True
10881,2023,2024,96,WRITING (Original Screenplay),Written by David Hemingson,The Holdovers,False
10882,2023,2024,96,WRITING (Original Screenplay),Written by Bradley Cooper & Josh Singer,Maestro,False
10883,2023,2024,96,WRITING (Original Screenplay),Screenplay by Samy Burch; Story by Samy Burch ...,May December,False
